In [14]:
# !!! ON HOLD - PDSTATE column creates duplication of values => breaks preprocessing pipeline
# probably wont have time to deal with, blocked for now



import pandas
import numpy

df_updrs_iii = pandas.read_csv('../source_data/MDS_UPDRS_Part_III.csv')
df_patnos = pandas.read_csv('../source_data/non_hc_patnos.csv')
df_sigfall = pandas.read_csv('../source_data/sigfall_main.csv')
non_hc_patnos = list(df_patnos.values.flat)

/tmp/ipykernel_14/220647287.py:4: DtypeWarning: Columns (10,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_updrs_iii = pandas.read_csv('../source_data/MDS_UPDRS_Part_III.csv')


In [17]:
# df_updrs_iii.query('PATNO == 3001').head(30)

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,PDMEDDT,PDMEDTM,PDSTATE,EXAMTM,DBS_STATUS,...,DYSKIRAT,NHY,PDTRTMNT,DBSONTM,DBSOFFTM,HRPOSTMED,HRDBSOFF,HRDBSON,ORIG_ENTRY,LAST_UPDATE
9,278743601,3001,BL,NUPDRS3,03/2011,NaN,NaN,NaN,13:05:00,NaN,...,NaN,1,0.0,NaN,NaN,NaN,NaN,NaN,03/2011,2021-07-31 00:00:00.0
10,274804301,3001,SC,NUPDRS3,02/2011,NaN,NaN,NaN,NaN,NaN,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,02/2011,2020-06-25 16:04:32.0
11,294967101,3001,V01,NUPDRS3,05/2011,NaN,NaN,NaN,08:50:00,NaN,...,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,06/2011,2020-06-25 16:04:33.0
12,310817501,3001,V02,NUPDRS3,08/2011,NaN,NaN,NaN,14:00:00,NaN,...,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,08/2011,2020-06-25 16:04:33.0
13,322772101,3001,V03,NUPDRS3,11/2011,NaN,NaN,NaN,08:25:00,NaN,...,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,12/2011,2020-06-25 16:04:33.0
14,341438201,3001,V04,NUPDRS3,03/2012,NaN,NaN,NaN,13:15:00,NaN,...,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,04/2012,2020-06-25 16:04:33.0
15,362683401,3001,V05,NUPDRS3,09/2012,09/2012,12:00:00,NaN,12:00:00,NaN,...,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,09/2012,2020-06-25 16:04:33.0
16,398204501,3001,V06,NUPDRS3,05/2013,04/2013,08:00:00,NaN,13:17:00,NaN,...,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,05/2013,2020-06-25 16:04:34.0
17,413438401,3001,V07,NUPDRS3,09/2013,09/2013,07:00:00,ON,08:08:00,NaN,...,NaN,2,1.0,NaN,NaN,1.1333,NaN,NaN,10/2013,2020-06-25 16:04:34.0
18,448713301,3001,V08,NUPDRS3,04/2014,04/2014,12:15:00,ON,09:47:00,NaN,...,NaN,2,1.0,NaN,NaN,0.7800,NaN,NaN,07/2014,2020-06-25 16:04:34.0


In [2]:
# --- Create dataframe that combines UPDRSiii data with SIGFALL & PREV_SIGFALL ---

def lookup_value(dframe,row,value):
    patno = row['PATNO']
    infodt = row['INFODT']
    
    s1 = dframe[(dframe.INFODT == infodt) & (dframe.PATNO == patno)][value]
    if s1.empty:    
        return numpy.nan
    else:
        return s1.values[0]
    
df_updrs_iii_sigfall_base = df_updrs_iii[df_updrs_iii['PATNO'].isin(non_hc_patnos)]
df_updrs_iii_sigfall_base = df_updrs_iii_sigfall_base.drop(['REC_ID','PAG_NAME', 'PDMEDDT',
       'PDMEDTM', 'PDSTATE', 'EXAMTM', 'DBS_STATUS','DYSKPRES', 'DYSKIRAT', 'PDTRTMNT','DBSONTM', 'DBSOFFTM','HRPOSTMED', 'HRDBSOFF', 'HRDBSON', 'ORIG_ENTRY',
       'LAST_UPDATE'], axis=1)
df_updrs_iii_sigfall_base['SIGFALL'] = df_updrs_iii_sigfall_base.apply (lambda row: lookup_value(df_sigfall,row,'SIGFALL'), axis=1)
df_updrs_iii_sigfall_base['PREV_SIGFALL'] = df_updrs_iii_sigfall_base.apply (lambda row: lookup_value(df_sigfall,row,'PREV_SIGFALL'), axis=1)
df_updrs_iii_sigfall_base = df_updrs_iii_sigfall_base.dropna()


In [3]:
# Compute Delta values, from last sample event

def get_infodts(dframe,patno):
    return list(dframe[dframe.PATNO == patno]['INFODT'].values)

def date_to_tuple(date):
    year = date[-4:]
    month = date[:2]
    return (month,year,date)

def date_sort(dates):
    tup_list = map(date_to_tuple,dates)
    return sorted(tup_list, key = lambda x: (x[1],x[0]))

def get_index(date,dates):
    output = (False,999)    
    for idx, val in enumerate(dates):
        if val[2] == date:
            output = (True,idx)
    return output

def prev_value(dframe,row,q):
    output = numpy.nan
    patno = row['PATNO']
    infodt = row['INFODT']    
    dates = get_infodts(dframe,patno)
    sorted_dates = date_sort(dates)
    i = get_index(infodt,sorted_dates)
    
    if i[0] == True:
        try:
            prev_date_index = i[1] - 1
            if prev_date_index > -1:
                prev_date = sorted_dates[prev_date_index][2]
                output = dframe[(dframe.INFODT == prev_date ) & (dframe.PATNO == patno)][q].values[0]
        except:
            pass
    return output

def q_delta_actual(dframe,row,q):
    output = numpy.nan
    patno = row['PATNO']
    infodt = row['INFODT']
    
    current = dframe[(dframe.INFODT == infodt ) & (dframe.PATNO == patno)][q].values[0]
    prev = prev_value(dframe,row,q)
    
    try:
        output = current - prev
    except:
        pass
    return output

updrs_qs = ['NP3SPCH', 'NP3FACXP',
       'NP3RIGN', 'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL', 'NP3FTAPR',
       'NP3FTAPL', 'NP3HMOVR', 'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR',
       'NP3TTAPL', 'NP3LGAGR', 'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT',
       'NP3PSTBL', 'NP3POSTR', 'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR',
       'NP3KTRML', 'NP3RTARU', 'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ',
       'NP3RTCON', 'NP3TOT', 'NHY']
dq_pairs = [(x,('D_'+x)) for x in updrs_qs]


for x in dq_pairs:
    df_updrs_iii_sigfall_base[x[1]] = df_updrs_iii_sigfall_base.apply (lambda row: q_delta_actual(df_updrs_iii_sigfall_base,row,x[0]), axis=1)


In [13]:
df_updrs_iii_sigfall_base.head(20)

,PATNO,EVENT_ID,INFODT,NP3SPCH,NP3FACXP,NP3RIGN,NP3RIGRU,NP3RIGLU,NP3RIGRL,NP3RIGLL,...,D_NP3KTRMR,D_NP3KTRML,D_NP3RTARU,D_NP3RTALU,D_NP3RTARL,D_NP3RTALL,D_NP3RTALJ,D_NP3RTCON,D_NP3TOT,D_NHY
27,3001,V15,03/2019,1,1,1,2,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,3001,V17,09/2021,2,2,3,2,2,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
30,3001,V17,09/2021,2,1,3,1,2,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
52,3002,V15,03/2019,0,1,0,1,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
53,3002,V15,03/2019,0,1,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
54,3002,V17,09/2021,1,1,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
55,3002,V17,09/2021,1,2,1,2,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
73,3003,V15,03/2019,1,1,0,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
74,3003,V15,03/2019,1,1,1,3,2,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
125,3010,V15,07/2019,3,2,0,1,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [11]:
# Extacting short form

# df_updrs_iii_short_form = df_updrs_iii_sigfall_base[['D_NP3TOT', 'D_NHY','PREV_SIGFALL', 'SIGFALL']]
# df_updrs_iii_short_form.dropna()
dq_pairs

[('NP3SPCH', 'D_NP3SPCH'),
 ('NP3FACXP', 'D_NP3FACXP'),
 ('NP3RIGN', 'D_NP3RIGN'),
 ('NP3RIGRU', 'D_NP3RIGRU'),
 ('NP3RIGLU', 'D_NP3RIGLU'),
 ('NP3RIGRL', 'D_NP3RIGRL'),
 ('NP3RIGLL', 'D_NP3RIGLL'),
 ('NP3FTAPR', 'D_NP3FTAPR'),
 ('NP3FTAPL', 'D_NP3FTAPL'),
 ('NP3HMOVR', 'D_NP3HMOVR'),
 ('NP3HMOVL', 'D_NP3HMOVL'),
 ('NP3PRSPR', 'D_NP3PRSPR'),
 ('NP3PRSPL', 'D_NP3PRSPL'),
 ('NP3TTAPR', 'D_NP3TTAPR'),
 ('NP3TTAPL', 'D_NP3TTAPL'),
 ('NP3LGAGR', 'D_NP3LGAGR'),
 ('NP3LGAGL', 'D_NP3LGAGL'),
 ('NP3RISNG', 'D_NP3RISNG'),
 ('NP3GAIT', 'D_NP3GAIT'),
 ('NP3FRZGT', 'D_NP3FRZGT'),
 ('NP3PSTBL', 'D_NP3PSTBL'),
 ('NP3POSTR', 'D_NP3POSTR'),
 ('NP3BRADY', 'D_NP3BRADY'),
 ('NP3PTRMR', 'D_NP3PTRMR'),
 ('NP3PTRML', 'D_NP3PTRML'),
 ('NP3KTRMR', 'D_NP3KTRMR'),
 ('NP3KTRML', 'D_NP3KTRML'),
 ('NP3RTARU', 'D_NP3RTARU'),
 ('NP3RTALU', 'D_NP3RTALU'),
 ('NP3RTARL', 'D_NP3RTARL'),
 ('NP3RTALL', 'D_NP3RTALL'),
 ('NP3RTALJ', 'D_NP3RTALJ'),
 ('NP3RTCON', 'D_NP3RTCON'),
 ('NP3TOT', 'D_NP3TOT'),
 ('NHY', 'D_NHY')]